<a href="https://colab.research.google.com/github/Luceliafn/Alura/blob/Machine-Learning-Introdu%C3%A7%C3%A3o-a-classifica%C3%A7%C3%A3o-com-SKLearn/Machine_Learning_Valida%C3%A7%C3%A3o_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Módulo 2 KFold e Aleatoriedade

Ainda que nosso algorítimo esteja funcionando, vamos revisar o código e nos atentar aos seus detalhes:

In [9]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados

x = dados [['preco', 'idade_do_modelo', 'km_por_ano']]
y = dados ['vendido']

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate

treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)


Treinaremos com 7500 elementos e testaremos com 2500 elementos


In [8]:
#aleatoriedade no cross validate
SEED = 301
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 5, return_train_score=False)
media = results ['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 5 = [%.2f, %.2f]" % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 5 = [75.21, 76.35]


O que desejamos modificar é a aleatoriedade, nós desejamos que ele embaralhe nossos dados, entretanto, não existe um parâmetro para isto. Todavia, o cv aceita como parâmetro, tanto números, quanto geradores de validação cruzada. E existem diversos geradores de validação cruzada. O que estamos usando é o KFold, que "corta" nossos dados em K pedaços. Por sorte, o sklearn tem o KFold, no modo selection:

class sklearn.model_selection.KFold(nsplits=’warn’, shuffle=False, randomstate=None)

É ele que estamos usando por baixo dos panos, ou seja, sorrateiramente. Então, por enquanto, nosso cross_validate é um KFold de n_splits, no caso, igual a 10. Vamos rodar o código, que com essas modificações, ficou da seguinte forma:

In [20]:
#def imprime_resultados(results):
 #   media = results['test_score'].mean()
 #  desvio_padrao = results['test_score'].std()
 #  print("Accuracy com cross validation = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao) * 100))

In [46]:
def imprime_resultados(results):
    media = results['test_score'].mean()
    desvio_padrao = results['test_score'].std()
    print("Accuracy médio: %.2f" % (media * 100))
    print("Accuracy intervalo: [%.2f][%.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao)*100))

Assim, ela recebe os results inteiros e, em seguida faz a média, o desvio padrão e imprime os resultado. Neste caso, podemos copiar e colar esta parte do código.

Para não dependermos de uma variável extra, excluímos 10 de print(). E, no código acima, adicionaremos uma linha ao fim do código, para que o algorítimo chame-se apenas imprime_resultados() de modo que não tenhamos de fazer manualmente sempre que formos rodar o código. Ele diretamente pegará a média, o desvio padrão e os imprimirá.

In [47]:
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
imprime_resultados (results)

Accuracy médio: 75.76
Accuracy intervalo: [73.26][78.26]


Como retorno, obteremos:

Accuracy com cross validation, 10 = [74.37, 77.19]COPIAR CÓDIGO
Agora, o ideal é pegarmos os valores da média de desvio padrão e colocarmos em uma função, para facilitarmos nosso trabalho. Nossa função será definida como:

Podemos subir o código de imprime_resultados() para cima, selecionando-o e pressionando o botão "CELL (com a seta para cima)", para que a definição venha em primeiro. A ordenação ficará da seguinte forma:

E rodemos o código logo acima novamente, da mesma forma que estávamos fazendo, com KFold de n_splits igual a 10. Como retorno, obteremos:

Accuracy com cross validation = [74.37, 77.19]


Repare como ficou difícil de acharmos a média. Por esta razão, vamos adicionar um print() para ela em imprime_resultados(). Para especificar, mudaremos os nomes dos prints para Accuracy médio e Accuracy intervalo, respectivamente.

É importante ressaltar que se rodarmos diversas vezes, Accuracy se mantém, pois continua sendo determinístico. Mas ainda assim, é diferente dos valores apresentados sem o shuffle. No entanto, repare que a média obtida por meio dos dois algorítimos é bastante próxima, mas o intervalo é diferente. Na prática, se os dados não possuem alguma estrutura interna, como uma sequência por datas, por exemplo, primeiro gera-se um sequência aleatória desses dados e, em seguida, quebra-se em 10 pedaços ou faz-se ambos ao mesmo tempo. Esta é a forma tradicional de execução com shuffle e n_splits.

Assim, podemos usar não somente o KFold, mas, em seu Model Selection, encontraremos o KFold e diversos outros "quebradores" de grupos, que são as Splitter classes. Estudaremos alguns destes outros adiante.

#Módulo 3 - Estratificação

Sempre podemos ter muito azar em uma condição muito específica. Antes de trabalharmos com validação cruzada, estudamos uma questão de sorte e azar em que a validação era simples: treino-teste. Tínhamos o azar de que nossos dados, no momento de realmente utilizá-los, eram separados como, por exemplo, vendido ou não vendido.

Como resultado, no treino estavam quase todos que não foram vendidos e no teste quase todos que foram vendidos. Ou seja, ocorria um desbalanceamento das classes, seja no conjunto inteiro de dados ou na hora de treinar e testar, e seja pela aleatoriedade ou pela natureza dos dados. Então, se tivermos azar, o que pode acontecer?

Quando separávamos o treino e o teste com o train_test_split — que também está no model_selection, em Splitter Functions — tínhamos o Stratify como parâmetro, que definia a proporção. Por exemplo, se temos 70% dos dados na classe 0 (não vendido) e 30% na classe 1 (vendido), ele distribui 70% e 30% tanto para treino, quanto para teste. No entanto, esse parâmetro não existe no cross_validate do KFold, que possui apenas os parâmetros n_splits, shuffle e random_state. Sendo assim, não está estratificando.

Então, vamos simular uma situação de azar para podermos aprimorar nosso processo. Sendo assim, identificaremos esse código com # Simular situação horrível de azar. Abaixo, adicionaremos o texto Pode ser uma situação de "azar" ou uma proporção de exemplos desbalanceado entre as classes. Pegaremos os dados (dados.sort_values()) do nosso data frame e ordenaremos pela classe vendido, crescente (ascending=True), ou seja, primeiro 0 e depois 1. Assim, o "não vendido" virá antes do "vendido". Essa é a separação que estabelecemos.

In [49]:
#Simular situação horrível de azar
#Pode ser uma situação de "azar" ou uma proporção de exemplos desbalanceado entre as classes.

dados.sort_values("vendido", ascending=True)

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704
...,...,...,...,...
5491,71910.43,1,9,25778.40812
1873,30456.53,1,6,15468.97608
1874,69342.41,1,11,16909.33538
5499,70520.39,1,16,19622.68262


Conseguimos ordenar de maneira que, primeiro, apareçam todos os veículos não vendidos (0) e posteriormente todos os vendidos (1), ambos em ordem crescente. Esses dados serão chamados de dados_azar. O x_azar será como o x anterior, composto por preco, idade_do_modelo e km_por_ano, enquanto y_azar será vendido. E pegaremos só os primeiros dados_azar, por meio de head().

In [50]:
dados_azar = dados.sort_values("vendido", ascending=True)
x_azar = dados_azar[["preco", "idade_do_modelo", "km_por_ano"]]
y_azar = dados_azar["vendido"]
dados_azar.head()

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


Agora, vamos rodar o KFold sem o shuffle, e com esses dados de azar.

In [51]:
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 57.84
Accuracy intervalo: [34.29][81.39]


In [54]:
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle=True)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.78
Accuracy intervalo: [72.30][79.26]


O suffle nos é suficientemente bom nesta situação em que temos balanço para recuperar, mas e se quiséssemos fazer a estratificação?

Além do KFold, temos outros geradores de grupo de separação, dentre eles o StratifiedKFold, que recebe um número de n_splits, shuffle e depois separa, mantendo a proporção entre as classes.

class sklearn.model_selection.StratifiedKFold(nsplits=’warn’, shuffle=False, randomstate=None)

É este que usaremos! Então, copiaremos e colaremos nosso algorítimo e, em cv, modificaremos KFold para StratifiedKFold, lembrando de importá-lo no início do código.

In [56]:
from sklearn.model_selection import StratifiedKFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.78
Accuracy intervalo: [72.30][79.26]


Embora o resultado seja mais fechado, é o mais recomendado. Principalmente quando há um desbalanço entre duas ou mais classes, é interessante utilizar o StratifiedKFold.

Reparem como ficariam os resultados se o deixassemos no aleatório. Inclusive, mesmo sem o shuffle, ele recupera bem a capacidade de prever o quão bom seria o treinamento. Vamos deletar shuffle:

In [57]:
from sklearn.model_selection import StratifiedKFold

SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.78
Accuracy intervalo: [73.83][77.73]


Feita essa demonstração, podemos voltar o shuffle ao código, mas será o StrafiedKFold que mais usaremos em casos de desbalanço entre as classes.

Como citado, existem diversas maneiras de validar os estimadores. A árvore de decisão DecisionTreeClassifier que criamos é uma delas, mas podíamos usar uma validação tradicional com treino-teste, com validação cruzada, que podem usar valores aleatórios para embaralhar os dados. Também podemos fazer isso de uma forma estratificada, ou misturando a estratificação com aleatoriedade e validação cruzada.

Reparem que, em sklearn.model_selection, encontramos diversas classes, cada uma em seu momento recomendado de uso. Existe muita literatura apontando os melhores momentos, vantagens e desvantagens de usar uma abordagem ou outra. Aqui, tento mostrar cenários onde é importante migrar de um para outro para termos uma previsão mais realista do quanto esse algorítimo pode ser bom em produção. Algumas dessas boas práticas são facilmente encontradas na própria documentação aqui do Sicikit Learn.

Então existem diversos guias de quando usar um e quando usa outro. Aqui, estamos pincelando alguns para que se tenha uma mínima noção apenas. À medida que você se aprofunda e aplica, não tenha medo de recorrer à documentação ou links externos, como artigos e tutorias.

A seguir, continuaremos com outras abordagens para que validemos nosso modelo, seja a fim de comparação ou para medir sua funcionalidade.

O que aprendemos nesta aula:

Simular uma situação com dados;
Ordenar nosso dataframe utilizando o sort_value(“vendido“’);

#Módulo 4 - Dados agupáveis

**Gravando dados Aleatórios**

Para continuarmos o curso, vamos retomar alguns pontos que abordamos até aqui.

Analisamos diversas maneiras de quebramos os dados para fazer a validação cruzada. Primeiramente, o train_test_split, que separava treino e teste e rodava os dados uma única vez. Depois nos atentamos ao KFold e ao StratifiedKFold com e sem aleatoriedade.

Vamos analisar uma situação cotidiana relacionada às características especiais de dados. Quando um paciente dá entrada em um hospital, a equipe retém diversas informações sobre ele. Assim, temos várias linhas de dados sobre o paciente Guilherme, diversas linhas de dados sobre a paciente Ana, e assim por diante.

É preciso tomar cuidado quando formos separar o treino e o teste. Temos que pensar que, ao treinarmos nosso algorítimo, nós treinamos ele com os dados; quando nós testamos, temos que ser capazes de testar como se fosse no mundo real. Entretanto, por exemplo, quando o Carlos entra no hospital, ele é um paciente novo e ainda não há dados sobre ele.

No mundo real em um hospital, ele é um paciente novo, nunca visto, e aquilo que fora treinado com outros pacientes precisa funcionar com ele também. Na hora de separarmos treino e teste, teríamos de agrupar pelos pacientes. Nós não poderíamos colocar dados da Ana no treino e no teste, porque cada paciente é uma pessoa nova, no mundo real do hospital. Claro, o paciente pode voltar, mas queremos ser capazes de trabalhar com pacientes novos. E se nos treinos e testes temos repetições de pacientes, é possível que o algorítimo não generalize bem, para novos pacientes.

A mesma coisa para os carros, vamos parar para pensar. Se nós temos diversos carros para os quais o nosso algorítimo, nosso estimador, funcionou bem no passado, quando surgir um novo modelo de carro, o algorítimo funcionará bem para esse último? Não sabemos! Do modo como estamos fazendo, treinamos todos os carros. No treino, nós temos todos os modelos de carro e no teste nós temos todos os modelos de carro aleatorizados e separado por classes, mas não agrupados por modelos. Ou seja, não agrupado pela característica que define o novo tipo de dado.

Então, vamos pegar essa característica de agrupamento e tentar trabalhar nos nossos dados. Ou seja, no caso dos carros, trata-se do modelo do veículo.

Até o momento, ainda não existe um modelo de carro chamado "Super Guilherme". Pode ser que esse modelo seja inventado daqui a 10 anos, por exemplo. Mas quando ele passar a existir, é preciso que sejamos capazes de prever para ele também. Para isto, no momento em que formos treinar, precisaremos agrupar por modelo e separar a validação cruzada por modelos, para depois testar modelos novos.

Contudo, nota-se que nossos dados (preco, vendido, idade_do_modelo e km_por_ano) não têm a categoria modelo_do_carro. Então, vamos explorar como gerar uma nova coluna de dados para explorarmos esses dados aleatórios.

É possível imaginar que o modelo de um carro esteja muito ligado a idade dele. Por exemplo, se um carro tem 20 anos, é mais provável que seja um modelo similar ao modelo de 18 anos atrás, do que ao modelo de 3 anos atrás. Isto posto, vamos usar o dado idade_do_modelo para gerar o modelo de um carro, de alguma maneira.

A maneira mais simples seria nós usarmos dados.idade_do_modelo, isto é, o modelo do carro seria a idade_do_modelo desse carro. Entretanto, essa é uma premissa falsa, uma vez que nem todos os carros que tem 20 anos necessariamente são do mesmo modelo. Em vez disso, temos que injetar aleatoriedade. Por exemplo, se o carro foi criado há 18 anos, talvez esse carro seja de um modelo A, B ou C. Um modelo do ano 18, 17 ou 19 — com margem um pouco para trás e um pouco para frente. Para gerar números aleatórios é comum o uso do NumPy (np), que tem random e foi por nós utilizado algumas vezes para setar o SEED.

Portanto, a primeira etapa é adicionar o SEED padrão para mantermos uma repetição. Abaixo, pegaremos um número inteiro (randint), considerando o ano do modelo menos dois (-2) até três (3).

In [58]:
np.random.seed(SEED)
np.random.randint(-2, 3)

-2

O randint gera um número entre -2 e 2. Se tirarmos a linha de SEED, será gerado várias vezes o 2, 0, e assim por diante. Ao ir à documentação do numpy.random.randint, encontramos essa aleatoriedade, que gera entre um mínimo (low) e um máximo (high).

numpy.random.randint(low, high=None, size=None, dtype='l')

Mas podemos apontar quantos números queremos gerar, neste caso, um número para cada um dos carros. O tamanho de nossos dados (len(dados)) é de 10.000 linhas. Então, este é o tamanho (size) que desejamos gerar.

In [61]:
np.random.seed(SEED)
np.random.randint(-2, 3, size=10000)

array([-2,  2,  0, ...,  0,  2,  2])

Desta forma, são gerados 10000 números entre -2 e 2 (3 é descartável). Agora, temos dez mil números aleatórios entre -2 e 2 e podemos somar (+) isso a dados.idade_do_modelo. Assim, teremos a idade do modelo, mais ou menos dois, para cada um dos carros. Então, o que antes era 18, passa a ser 16; o que antes era 20, passa a ser 22, e assim por diante. Aplicaremos essa lógica na nova coluna (modelo_aleatorio), para ver os campos lado a lado e, em seguida, chamamos apenas os primeiros elementos.

In [62]:
np.random.seed(SEED)
dados["modelo_aleatorio"] = dados.idade_do_modelo + np.random.randint(-2, 3, size=10000)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio
0,30941.02,1,18,35085.22134,16
1,40557.96,1,20,12622.05362,22
2,89627.50,0,12,11440.79806,12
3,95276.14,0,3,43167.32682,4
4,117384.68,1,4,12770.11290,3


Notem que, na primeira linha, temos 16 na coluna de modelo_aleatorio, sendo que o valor é 18 em idade_do_modelo, ou seja uma diferença de -2. Já na segunda linha, a diferença é de +2, pois temos 22 em modelo_aleatorio e 20 em idade_do_modelo. Na terceira linha, a diferença é de 0, considerando que tanto em idade_do_modelo quanto em modelo_aleatorio o valor é 12. Seguindo as comparações, ainda encontramos diferenças de +1 e -1, e por aí vai.

Com isso, percebemos que, agora, temos um pouco de aleatoriedade, de acordo com a idade_do_modelo. Isso será suficiente para trabalharmos com modelos de carros. Uma única característica em que queremos destacar são os modelos gerados. Analisemos este modo aleatório, de forma única (unique()), executando o seguinte comando:

In [63]:
dados.modelo_aleatorio.unique()

array([16, 22, 12,  4,  3, 11, 18, 17, 13,  0, 15, 10,  9, 14,  1,  5, 19,
       21,  8,  7, 20,  6,  2, -1])

Parece fazer sentido. No entanto, o modelo -1 ficou esquisito. Provavelmente porque era um valor 0 que, menos um, ficou -1. Uma maneira mais simples de retirar este número, é pegar o menor de todos esses valores, ou seja, o minimo deles: min:

In [64]:
dados.modelo_aleatorio.min()

-1

Portanto, transformamos esse valor mínimo (-1) em 0, somando (+) 1. Pegaremos o valor absoluto deste número (abs) — que é 1:

In [65]:
abs(dados.modelo_aleatorio.min())

1

E somaremos em toda coluna de modelo_aleatorio. Então, já que a coluna já existe, podemos somar modelo_aleatório a abs() com:

In [66]:
dados.modelo_aleatorio = dados.modelo_aleatorio + abs(dados.modelo_aleatorio.min())
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio
0,30941.02,1,18,35085.22134,17
1,40557.96,1,20,12622.05362,23
2,89627.50,0,12,11440.79806,13
3,95276.14,0,3,43167.32682,5
4,117384.68,1,4,12770.11290,4


Agora sim, analisarmos o head(), com os cinco primeiros valores da tabela. Novamente, chamaremos min() para verificar se o menor valor é realmente 0:

In [67]:
dados.modelo_aleatorio.min()

0

Pode ser incômodo que o valor mínimo seja 0, então vamos transformá-lo em 1, somando um (+ 1) ao final de modelo_aleatorio, após abs():

In [68]:
dados.modelo_aleatorio = dados.modelo_aleatorio + abs(dados.modelo_aleatorio.min()) + 1
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,5


Então, geraremos novamente os números com o SEED, executando:

In [69]:
np.random.seed(SEED)
dados['modelo_aleatorio'] = dados.idade_do_modelo +np.random.randint(-2, 3, size=10000)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio
0,30941.02,1,18,35085.22134,16
1,40557.96,1,20,12622.05362,22
2,89627.50,0,12,11440.79806,12
3,95276.14,0,3,43167.32682,4
4,117384.68,1,4,12770.11290,3


E, abaixo, executando unique(), mostremos que há diversos valores, o mínimo continua sendo - 1 e o absoluto sendo 1. No entanto, na sequência, executaremos o código ao qual somamos um (+ 1) à coluna de modelo_aleatorio:

In [70]:
dados.modelo_aleatorio = dados.modelo_aleatorio + abs(dados.modelo_aleatorio.min()) + 1
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,5


In [71]:
dados.modelo_aleatorio.min()

1

Deletaremos as célula de abs() e unique(), porque elas não serão mais necessárias. Para isso, podemos selecioná-las e acionar o comando "Command + M D" ou clicar nos três pontos enfilerados na vertical, localizados no canto direito da célula que queremos deletar e selecionar a opção "Delete cell".

Vamos focar em gerar (SEED), somar um (+ 1) à coluna de modelo_aleatorio e imprimir. Como estamos executando os processos de uma vez só, podemos juntá-los. Então, selecionaremos, copiaremos e colaremos o código de soma ao SEED, deixando-o da seguinte forma:

In [74]:
np.random.seed(SEED)
dados['modelo_aleatorio'] = dados.idade_do_modelo + np.random.randint(-2, 3, size=10000)
dados.modelo_aleatorio = dados.modelo_aleatorio + abs(dados.modelo_aleatorio.min()) + 1
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,5


Se rodarmos unique() novamente:

In [75]:
dados.modelo_aleatorio.unique()

array([18, 24, 14,  6,  5, 13, 20, 19, 15,  2, 17, 12, 11, 16,  3,  7, 21,
       23, 10,  9, 22,  8,  4,  1])

Agora que geramos os dados que gostaríamos, mudaremos o nome do campo modelo_aleatorio para simplesmente modelo e faremos as devidas substituições nos locais em que modelo_aleatorio aparecia no código. Como já tínhamos a coluna modelo_aleatorio, ela se mantém. Para obter os valores únicos, bastará executarmos unique():



In [77]:
dados.modelo_aleatorio.unique()

array([18, 24, 14,  6,  5, 13, 20, 19, 15,  2, 17, 12, 11, 16,  3,  7, 21,
       23, 10,  9, 22,  8,  4,  1])

E para contar quantas vezes cada valor de modelo apareceu, bastará executarmos:

In [78]:
dados.modelo_aleatorio.value_counts()

20    901
19    798
18    771
21    723
17    709
16    668
14    621
22    575
15    573
13    557
12    511
11    401
10    371
23    370
9     336
8     278
7     206
24    199
6     181
5     108
4      76
3      44
2      17
1       6
Name: modelo_aleatorio, dtype: int64

Nela, encontramos que o modelo número 20 apareceu 901 vezes, o modelo 19 apareceu 798, e assim por diante. Analisando-a, perceberemos que alguns modelos aparecem mais que outros. Maravilha! Era o que queríamos.

Se quisermos remover a coluna modelo_aleatorio, na barra de menu superior, podemos usar o caminho "Runtime > Restart and run all", para que ele rode do zero e apareça somente a coluna modelo.

**Validação cruzada usando grupos**

Agora que temos os dados criados, como modelo do carro, precisamos testar nosso estimador. Para isto, lembrem-se, que é muito importante pensar qual tipo de teste e treino será utilizado, pois se escolhermos um tipo inapropriado, obteremos uma estimativa irreal de como esse estimador funcionará no mundo real.

Por exemplo, se escolhermos um StratifiedKFold, com shuffle:

In [79]:
from sklearn.model_selection import StratifiedKFold

SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.78
Accuracy intervalo: [73.55][78.01]


Os agrupamentos serão ignorados. E, embora ele funcione bem, os modelos de carro que aparecerão tanto em treino quanto em teste não serão realistas. No mundo real, a qualquer momento pode surgir um novo modelo de carro e nosso algorítimo terá que saber lidar com esse modelo. Contudo, a Accuracy de 75.78, obtida na execução do código acima, não condiz necessariamente com adição de um modelo novo, porque ele usa todos os modelos no treino e no teste — provavelmente por causa do shuffle —, o que é irreal.

Sendo assim, em vez de utilizar o StratifiedKFold com shuffle, vamos usar o tradicional KFold:



In [82]:
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 57.84
Accuracy intervalo: [34.29][81.39]


No entanto, este modelo se vale de todos os modelos de carro para prever todos os modelos de carro, portanto, também é irreal. O que realmente precisamos é: de capacidade para separar treino e teste na validação cruzada, de acordo com os grupos. Se formos ao scikit-learn, encontraremos validação cruzada que separa por grupo. Para encontrá-lo, digite cross validation group kfold em sua ferramenta de busca, e acesse o resultado que contém "sklearn.model_selection.GroupKFold — scikit-learn 0.20.0 ...".

Copiemos e colemos o código StratifiedKFold e o colaremos em uma nova célula, ao final, para alterarmos o model_selection de StratifiedKFold para GroupKFold, e substituindo StratifiedKFold por ele em todas as linhas que for necessário. Manteremos o número de splits (n_splits) igual a 10 e removeremos o shuffledeixando o algorítimo da seguinte forma:

In [89]:
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
#results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, groups = dados.modelo_aleatorio, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.78
Accuracy intervalo: [73.67][77.90]


O valor da média obtida é próximo ao de StratifiedKFold, porque os dados de um modelo de carro estão funcionando de maneria muito parecida, mas isso não é regra. Por exemplo, imagine que passe a existir um carro voador que faz viagens espaciais. A maneira de precificar este carro será totalmente diferente do que este estimador é capaz de classificar.

No caso que estamos trabalhanso, de modo geral, os carros se comportam indiferentemente do modelo. Então, nossa classificação foi próxima, levando em consideração ou não o modelo do carro. Quando é dito "levando em consideração" não é para estimar, mas sim para treinar e testar. Reparem que x o x_azar, entre outros, continuam sem usar a coluna modelo. Para classificar só usamos preco, idade_do_modelo e km por ano.

Este é o exemplo do GroupKFold e do porquê é importante o utilizarmos. Adiante, estudaremos outras maneiras de usar a seleção de modelos.



**O que aprendemos nesta aula:**

* Gerar dados aleatórios com o randint;
* Gerar uma nova coluna de dados;
* Utilizar o comando unique para mostrar os modelos únicos;
* Utilizar o GroupKFold para agrupar por grupo.


#Modulo 5 - Pipeline de treino e validação

**A importância do pipeline no crossvalidate**

A seguir, vamos organizar o que fizemos até agora. Como geramos alguns dados de modelo, intitularemos essa ação, clicando em "+ TEXT", no menu superior. Assim, adicionaremos uma caixa de texto, na qual digitaremos #Gerando dados aleatórios de modelo de carro para simulação de agrupamento ao usar nosso estimador. Feito isso, o título será encontrado em destaque, acima do respectivo trecho de código:

In [91]:
np.random.seed(SEED)
dados['modelo'] = dados.idade_do_modelo + np.random.randint(-2, 3, size=10000)
dados.modelo = dados.modelo + abs(dados.modelo.min()) + 1
dados.head

<bound method NDFrame.head of           preco  vendido  ...  modelo_aleatorio  modelo
0      30941.02        1  ...                18      18
1      40557.96        1  ...                24      24
2      89627.50        0  ...                14      14
3      95276.14        0  ...                 6       6
4     117384.68        1  ...                 5       5
...         ...      ...  ...               ...     ...
9995   97112.86        0  ...                14      14
9996  107424.63        1  ...                17      17
9997   93856.99        0  ...                 6       6
9998   51250.57        1  ...                11      11
9999   87945.73        0  ...                23      23

[10000 rows x 6 columns]>

Mais à frente, quando executamos o teste, intitularemos o seguinte código como #Testando validação cruzada com GroupKFold:

In [93]:
#Testando validação crizada com GroupKFold

from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, groups = dados.modelo, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.78
Accuracy intervalo: [73.67][77.90]


Pronto! Daqui para frente, a ideia é lidar com outro tipo de estimador com validação cruzada.

Anteriormente, percebemos que a árvore de decisão não precisa primeiramente balancear, escalar, as nossas features. Alguns algorítimos precisam, como o SVM, visto no curso anterior. Eles são muito influenciáveis e sensíveis à escala de cada uma das features. Então, analisando o resultado obtido de sort_values(), com ascending=True:



Se idade_do_modelo tem valores entre 0 e 20 e a preco tem entre 0 e 90 mil, a influência dessas features em nosso algorítimo, no SVM, por exemplo, será muito diferente entre uma e outra. Então, reescalamos as features quando usamos os algorítimos que são sensíveis à escala, à grandeza dos dados.

Ao final de tudo que codamos e executamos, adicionaremos uma caixa de texto, clicando em "+ TEXT", para adicionar o título #Cross validation com StandardScaler, que é o escalador que havíamos utilizado anteriormente.

Da mesma forma que fizemos anteriormente, utilizaremos o Standardscaler na parte de pré processamento. Vamos importá-lo com:

In [97]:
from sklearn.preprocessing import StandardScaler

Lembram que já havíamos separado o x e o y, com train_test_split? Rodaremos essa variação mesmo com o SVC. Então, pegaremos treino e teste e criaremos o scaler primeiro, que é o SatandardScaler. O scaler será treinado (fit()) no treino_x e depois vamos transformar (transform()) treino_x e teste_x. O primeiro será chamando de treino_x_escalado e o segundo será teste_x_escalado.

Após escalar, rodaremos o modelo de SVC, ou svm, do SKLearn, que utilizamos logo no começo. Copiaremos o código para fazer o import, abaixo do import de StandardScaler. Abaixo de teste_x_escalado, definiremos o modelo como SVC. Feito isso, rodaremos o fit(), gerar as previsoes e accuracy_score, da mesma forma que fizemos anteriormente. Sendo assim, copiaremos as respectivas linhas e as colaremos abaixo de modelo. Lembrem-se de fazer os devidos ajustes! Em fit(), trabalharemos as versões escaladas de treino_x e teste_x. E, agora que o SVC tem a questão da aleatoriedade, vamos adicionar o SEED igual a 301. O código ficará da seguinte forma:



In [101]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
scaler.fit(treino_x)
treino_x_escalado = scaler.transform(treino_x)
teste_x_escalado = scaler.transform(teste_x)

modelo = SVC()
modelo.fit(treino_x_escalado, treino_y)
previsoes = modelo.predict(teste_x_escalado)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A Acurácia foi de %.2f" % acuracia)


A Acurácia foi de 77.52


Maravilha! Agora, vamos rodar treino-teste de cross_validate com agrupamento (GroupKFold). E, em vez de usar DecisionTreeClassifier, usaremos o SVC(). Por não fazer parte dele, max_depth precisa ser retirado. Então, o treino-teste que queremos rodar ficará da seguinte forma:



In [102]:
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = SVC()
results = cross_validate(modelo, x_azar, y_azar, cv = cv, groups = dados.modelo,return_train_score=False)
imprime_resultados(results)

Accuracy médio: 77.27
Accuracy intervalo: [74.35][80.20]


Ao rodar, talvez após um tempo um pouco maior, obteremos um péssimo resultado:

Accuracy médio: 58.00
Accuracy intervalo: [56.10, 59.89]

Se pensarmos, será que os dados escalados foram utilizados? Ao analisarmos, percebeeremos que não, porque os dados escalados estão em treino_x_escalado e teste_x_escalado e, no caso, em cross_validate() foi utilizado o x_azar, que não está escalado e faz parte dos dados crus.

Para escalar e inseri-los em cross_validate(), vamos reescalar todo o x_azar e utilizá-lo de uma só vez. Antes, vamos pensar mais um pouquinho. Para escalar o x_azar, teremos que criar um StandardScaler, dar um fit() e um transform() nele. Assim, estamos fazendo o scaling do x_azar:

In [104]:
scaler = StandardScaler()
scaler.fit(x_azar)
x_azar_escalado = scaler.transform(x_azar)

In [105]:
x_azar_escalado

array([[ 0.3636103 , -0.39591706,  0.24651152],
       [ 0.79214444, -0.18257669,  0.03346607],
       [ 0.72310419,  1.09746554,  1.66435195],
       ...,
       [ 0.17822399, -0.60925743, -0.73405662],
       [ 0.2248779 ,  0.45744443, -0.39741686],
       [-1.34266351,  0.88412517,  1.52099053]])

Agora sim, valores que estão escalados, tirando um desvio padrão e a média, dividindo pelo desvio padrão, e temos os valores escalados. Então, rodaremos novamente o cross_validate com GroupKFold, com x_azar_escalado em vez de x_azar. Rodemos o código da seguinte forma:

In [106]:
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = SVC()
results = cross_validate(modelo, x_azar_escalado, y_azar, cv = cv, groups = dados.modelo,return_train_score=False)
imprime_resultados(results)

Accuracy médio: 76.70
Accuracy intervalo: [74.30][79.10]


Após um tempo, receberemos o seguinte resultado, que foi bem melhor:

Accuracy médio: 76.71
Accuracy intervalo: [74.30, 79.12]

Entretanto, no pré processamento de dados, quando escalamos as features, fit() é aplicado somente nos dados de treino, e aqui o fit() acontece em treino e teste. Até porque faremos dez separações baseadas nos grupos, para treinar e testar. Ou seja, não faremos o fit() somente uma vez. Devemos fazer esse fit() diversas vezes. Na verdade, deveríamos intercalar, fazendo uma vez esse fit, uma vez a validação, uma vez o fit, outra vez a validação, com outro conjunto de dados.

Toda vez que rodamos o GroupKFold, temos que fazer o fit() com uma parte de treino e rodar a validação com o teste. Se tivermos outro treino, rodaremos fit() para esse outro treino, e o teste com o resto. Assim, teremos um processo de duas fases:

Primeiro, o pré processamento que escala;
depois, a validação.
Esse processo deve ser rodado várias vezes, de acordo com os nossos grupos e com nossos com nossos splits. Então, precisamos pensar em outra forma, além de fazer o fit() somente uma vez e, depois, rodar o GroupKFold. Provavelmente, a estimativa obtida por esse processo está muito otimista.

Sendo assim, no Sklearn, criaremos um processo, uma sequência de passos, chamada Pipeline, que está em um módulo separado. Começaremos importando-o com:

In [107]:
from sklearn.pipeline import Pipeline

O Pipeline() será como um processo de dois passos

o primeiro é o scaler;
depois, vem o modelo.
Assim, scaler será igual ao StandarScaler(), e o modelo igual ao SVS().

In [109]:
from sklearn.pipeline import Pipeline

scaler = StandardScaler()
modelo = SVC 

pipeline = Pipeline(scaler, modelo)

TypeError: ignored

Reparem que não foi feito o fit(). Apenas criamos o scaler e o modelo. Podemos passá-los dentro de um array ([]), com os nomes transformação e estimador, respectivamente.



In [110]:
from sklearn.pipeline import Pipeline

scaler =StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)])
pipeline

Pipeline(memory=None,
         steps=[('transformacao',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('estimador',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='rbf', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

Desse modo, se imprimirmos ou analisarmos por dentro, encontraremos: primeiro uma transformação, que é o Standrscaler(), e depois um estimador que é um SVC().

O legal do Pipeline é que ele funciona como se fosse um estimador, então tem o fit() e o predict, que funciona da mesma forma. Então, podemos passar ele para o processo de cross_validate. Portanto, copiaremos o seguinte trecho do cross_validate com GroupKFold e x_azar_escalado:

In [111]:
SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = SVC()
results = cross_validate(modelo, x_azar_escalado, y_azar, cv = cv, groups = dados.modelo,return_train_score=False)
imprime_resultados(results)

Accuracy médio: 76.70
Accuracy intervalo: [74.30][79.10]


E o colaremos em nosso Pipeline. Por padrão, organizaremos, mantendo SEED no começo. Assim, criaremos o scaler e o SVC, e os utilizaremos em pipeline. Na sequência, definimos GroupKFold() e aplicamos cross_validate() para o pipeline inteiro, utilizando o exemplo de x_azar.

In [112]:
from sklearn.pipeline import Pipeline

SEED = 301
np.random.seed(SEED)

scaler =StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)])

cv = GroupKFold(n_splits = 10)
result = cross_validate(pipeline, x_azar, y_azar, cv = cv, groups = dados.modelo, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 76.70
Accuracy intervalo: [74.30][79.10]


E vamos rodar esse código. Agora sim, para cada um dos processos de fold, serão rodados tanto a transformacao, quanto o estimador. Depois, outro conjunto de dados será separado em dois, tanto a transformacao, quanto o estimador. Portanto, a execução do código vai demorar um pouco mais. O importante é que teremos uma estimativa mais realista do modelo SVC(), com StandardScaler, no mundo real, considerando agrupamentos. Vide resultado:

Accuracy médio: 76.68
Accuracy intervalo: [74.28, 79.08]

Reparem que o valor anterior e este não são tão distantes, mas é preciso entender que a forma feita anteriormente estava errada, porque rodávamos o scaler somente uma vez, para depois rodar os KFolds. Não é questão de estar melhor ou pior, pois estamos falando de um processo, que deve ser rodado para cada uma das fases de treino. Portanto, se tivermos 10 fases de treino, teremos que rodar o scaler para cada uma do conjunto de treino. E é isso que o Pipeline faz para nós, de forma muito mais prática. Basta inserir todas as fases nele e rodar. Claro, pode demorar um pouco mais, mas é a opção que faz mais sentido.

Para rodar algorítimos com cross_validate, que exigem pré processamento, vocês talvez queiram fazer o treino desse pré processamento somente na parte de treino. Para isso será necessário incluí-lo em um Pipeline.

**Treinando o modelo final**

Agora que temos diversas abordagens de validação você vai escolher a que acredita ser ideal para as características dos seus dados, do seu dataset, de acordo com o que deseja predizer.

A média e o intervalo providos pela validação cruzada diz o quão bem você espera que o modelo se comporte com dados previamente desconhecidos mas... se você usou cross validation com 10 folds, qual dos 10 modelos treinados você vai usar agora no mundo real?

A ideia é que a validação cruzada num conjunto de dados somente te diz o que você pode esperar. Ela não treina o seu modelo final.

Agora você tem o algoritmo do estimador (por exemplo um DecisionTreeClassifier) e pretende usá-lo no mundo real:

In [ ]:
modelo = DecisionTreeClassifier(max_depth = n)
resultados = cross_validate(modelo, x, y, cv = ...)

Resultados indica o que esperar. Para obter o modelo final basta treiná-lo em todos os dados que você usou para o crossvalidation:

In [ ]:
modelo.fit(x, y)

Pronto, seu modelo está pronto para ser usado com o predict para classificar etc dados novos.



O que aprendemos nesta aula:

Treinar usando o módulo SVC
O que é pipeline
Utilizar o pipeline